In [32]:
import matplotlib.pyplot as plt
import pickle
import os
import snap
from itertools import combinations
import numpy as np
from google.cloud import bigquery
import collections
from IPython.lib import backgroundjobs as bg
from tqdm import tqdm_notebook
import heapq
from gensim.models import Word2Vec
import node2vec
import networkx as nx

In [33]:
with open('./data/project_authors_2016_q1_k25core.pickle', 'rb') as picklefile:
    project_authors_q1 = pickle.load(picklefile)

In [36]:
dimensions=32
window=10
iterations=10
p=0.1
q=10.
num_walks=25
walk_length = 400

In [15]:
nx_weighted = nx.Graph()
nx_weighted.add_nodes_from(project_authors_q1['author_id'].unique())
nx_weighted.add_nodes_from(project_authors_q1['project_id'].unique())
for _, row in project_authors_q1.iterrows():
    nx_weighted.add_edge(-row['project_id'], row['author_id'], weight=row['commit_times'])

In [16]:
len(nx_weighted)

7376

In [29]:
G = node2vec.Graph(nx_weighted, False, p, q)
%time G.preprocess_transition_probs()

CPU times: user 3min 17s, sys: 1.21 s, total: 3min 18s
Wall time: 3min 18s


In [37]:
%time walks = G.simulate_walks(num_walks, walk_length)
walks = [map(str, walk) for walk in walks]
%time model = Word2Vec(walks, size=dimensions, window=window, min_count=0, sg=1, iter=iterations)

Walk iteration:
1 / 25
2 / 25
3 / 25
4 / 25
5 / 25
6 / 25
7 / 25
8 / 25
9 / 25
10 / 25
11 / 25
12 / 25
13 / 25
14 / 25
15 / 25
16 / 25
17 / 25
18 / 25
19 / 25
20 / 25
21 / 25
22 / 25
23 / 25
24 / 25
25 / 25
CPU times: user 21min 47s, sys: 6.92 s, total: 21min 54s
Wall time: 22min
CPU times: user 36min 5s, sys: 12.7 s, total: 36min 18s
Wall time: 12min 12s


In [38]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)
with open('./data/2016_q2_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q2 = pickle.load(picklefile)
edges_q1 = set(edge for edge, weight in weights_q1)
edges_q2 = set(edge for edge, weight in weights_q2)
new_edges=edges_q2.difference(edges_q1)
k = len(new_edges)

0.1 10 600 25 (943, 0.037240344364584156)
0.1 10 100 100 (2767, 0.1092725693073217)
10 10 600 25 (2840, 0.11215543795908696)
10 10 100 100 (3157, 0.12467419635099913)

In [39]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

(3310, 0.1227244076971562)

In [21]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

(2840, 0.11215543795908696)

In [40]:
pairs = list(combinations(authors, 2))
alldist = {}
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    alldist[(u, v)] = model.wv.distance(str(u), str(v))
with open('./data/node2vec-weighted-p01-q10-25-400.pickle', 'wb') as picklefile:
    pickle.dump(alldist, picklefile)